# Homework 2: YTM, Spot Rate, and Forward Rate

## 前言：

第二次作業要實作三項指標：

- Yield to Maturity (YTM)
- Spot Rate
- Forward Rate

## YTM

首先，我們的目標是實作出如 [Bond Yield Calculator](https://www.calkoo.com/en/ytm-calculator) 一樣的 YTM 計算機  
一開始先讓使用者輸入計算 YTM 所需要的參數：

In [1]:
P = float(input("Current Bond Price: "))
F = float(input("Bond Par Value: "))
c = float(input("Bond Coupon Rate(%): ")) / 100
n = float(input("Years to Maturity: "))
m = int(input("Payment(1. Annually, 2. Semi-annually, 4. Quarterly): "))

Current Bond Price:  950
Bond Par Value:  1000
Bond Coupon Rate(%):  12
Years to Maturity:  3
Payment(1. Annually, 2. Semi-annually, 4. Quarterly):  2


計算 YTM 有兩種做法：

- 第一種是上課提到的做法，利用現有的套件，例如 Excel 的 Solver 去解方程式，而 Python 中也有類似的套件：SymPy
- 第二種做法是窮舉 YTM，根據計算出來的 Bond Price 再不斷逼近真實的 YTM  

以下會採取第二種做法

1. 第一步我們需要一個計算 Bond Price 的 function，參考該網站 [Bond Price Calculator](https://www.thecalculator.co/finance/Bond-Price-Calculator-606.html) 我們可以實作出以下的 function

In [2]:
def compute_p(F, c, m, n, r):
    C = F * c / m
    P = C * (1 - (1 + r/m)**(-n*m)) / (r/m) + F / (1 + r/m)**(n*m)
    return P

2. 而窮舉 YTM 的數值雖然不難，但我們可以有更聰明的作法，根據 [YTM Approximation Formul](https://financeformulas.net/Yield_to_Maturity.html) 我們可以實作出以下的 function

In [3]:
def approximate_r(P, F, c, m, n):
    r = ((F*c/m) + (F-P)/n) / ((F+P)/2)
    return r

3. 最後，使用以上兩個 functions，就可以針對 YTM 進行估計

In [4]:
def compute_r(P, F, c, m, n):
    r = approximate_r(P, F, c, m, n) # Initialize r by approximation    
    inc = 0.01
    
    while compute_p(F, c, m, n, r) - P > 0.0001: # If the derived P is larger than actual P, we should increase r
        if compute_p(F, c, m, n, r+inc) < P:  # If the derived P is smaller than actual P, we should lower the amount added to r
            inc /= 10
        else:
            r += inc          
    while P - compute_p(F, c, m, n, r) > 0.0001: # If the derived P is small than actual P, we should decrease r
        if compute_p(F, c, m, n, r-inc) > P:  # If the derived P is larger than actual P, we should lower the amount minus to r
            inc /= 10
        else:
            r -= inc

    if r < 0:
        r = 0
    return r/m 

注意：計算出來的 YTM 並未進行年化

In [5]:
r = compute_r(P, F, c, m, n)
print(f"Yield to Maturity: {r*100:.4f}%")

Yield to Maturity: 7.0506%


## Spot Rate

關於 Spot Rate 的計算，根據使用者輸入的資訊不同，計算方法也會不同， 以下實作兩種算法：  
第一種：[Trignosource: Spot Rate](https://www.trignosource.com/finance/spot%20rate.html)   
-> 使用者輸入 Zero Coupon Bond Price 與其 Duration，計算 Spot Rate

In [6]:
t = float(input("Duration of spot rate: "))
p = float(input("Unit zero-coupon bond price(0-1): "))

Duration of spot rate:  4
Unit zero-coupon bond price(0-1):  0.683


In [7]:
s = p ** (-1 / t) - 1
print(f"{t} year spot rate of interest: {s*100:.2f}%")

4.0 year spot rate of interest: 10.00%


第二種：[綠角財經筆記](http://greenhornfinancefootnote.blogspot.com/2010/06/how-to-compute-theoretical-spot-rates.html)  
-> 使用者輸入各期資訊， 計算 Spot Rate

1. 首先要知道**總共持續多少時間**與**多久計算一次利息**，才有辦法決定接下來要計算多少資訊

In [8]:
n = float(input("Years to Maturity: "))
m = int(input("Payment(1. Annually, 2. Semi-annually, 4. Quarterly): "))

Years to Maturity:  1.5
Payment(1. Annually, 2. Semi-annually, 4. Quarterly):  2


2. 根據以上資訊，請使用者輸入足夠的資訊 (Bond Price, Par Value, Coupon Rate)，再使用前面定義的 function 計算出個期的 YTM

In [9]:
coupon_rate = list()
ytm = list()
for i in range(int(m * n)):
    P = float(input(f"Bond Price of a {(i+1) / m} year due bond: "))
    F = float(input(f"Par Value of a {(i+1) / m} year due bond: "))
    c = float(input(f"Coupon Rate of a {(i+1) / m} year due bond: ")) / 100
    coupon_rate.append(c)
    r = compute_r(P, F, c, m, (i+1) / m)
    ytm.append(r * m)
    print()

Bond Price of a 0.5 year due bond:  995
Par Value of a 0.5 year due bond:  1000
Coupon Rate of a 0.5 year due bond:  0


Bond Price of a 1.0 year due bond:  980.3
Par Value of a 1.0 year due bond:  1000
Coupon Rate of a 1.0 year due bond:  0


Bond Price of a 1.5 year due bond:  1029.1
Par Value of a 1.5 year due bond:  1000
Coupon Rate of a 1.5 year due bond:  5


3. 計算 Spot Rate 的過程，一開始是計算 Present Value，之後再換算回 Zero Coupon Bond 的組合。這個過程可以化簡為以下的 function

In [10]:
def compute_spot(spot_rates, m, ytm, c):
    pv = 0
    for i in range(len(ytm)):
        if i != (len(ytm) - 1):
            pv += (c / 2) / ((1 + ytm[len(ytm)-1] / 2) ** (i + 1))
            pv -= (c / 2) / ((1 + spot_rates[i] / m) ** (i + 1))
        else:
            pv += (1 + c / 2) / ((1 + ytm[len(ytm)-1] / 2) ** (i + 1))
            sn = ((pv / (1 + c / 2)) ** (-1 / (i + 1)) - 1) * 2
            return sn

4. 有了足夠的資訊與工具，便能依序計算出個期的 Spot Rate

In [11]:
spot_rates = list()
for i in range(int(m * n)):
    if (i+1) / m <= 1:
        spot_rates.append(ytm[i])
    else:
        spot_rates.append(compute_spot(spot_rates, m, ytm, coupon_rate[i]))

In [12]:
import pandas as pd
from tabulate import tabulate

due_row = [f"{(i+1)/m} 年" for i in range(int(m * n))]
ytm_row = [f"{round(i*100, 2)}%" for i in ytm]
spot_rates_row = [f"{round(i*100, 2)}%" for i in spot_rates]

spot_rate_matrix = [due_row, ytm_row, spot_rates_row]

spot_rate_table = pd.DataFrame(spot_rate_matrix, index=['到期時間', 'YTM', 'Spot Rate'])
print(tabulate(spot_rate_table, tablefmt="grid"))

+-----------+--------+--------+--------+
| 到期時間  | 0.5 年 | 1.0 年 | 1.5 年 |
+-----------+--------+--------+--------+
| YTM       | 1.01%  | 2.0%   | 3.0%   |
+-----------+--------+--------+--------+
| Spot Rate | 1.01%  | 2.0%   | 3.04%  |
+-----------+--------+--------+--------+


## Forward Rate

關於 Forward Rate 的計算，同樣有兩種算法：  
第一種：[Trignosource: Forward Rate](https://www.trignosource.com/finance/Forward%20rate.html)，只計算單一的 Forward Rate

In [13]:
t = int(input("Time due for the beginning of forward rate(year): "))
r = int(input("Duration of forward rate(year): "))
pt = float(input(f"The price of the {t} year zero-coupon bond per unit nominal: "))
ptr = float(input(f"The price of the {t+r} year zero-coupon bond per unit nominal: "))

Time due for the beginning of forward rate(year):  3
Duration of forward rate(year):  2
The price of the 3 year zero-coupon bond per unit nominal:  0.83
The price of the 5 year zero-coupon bond per unit nominal:  0.6743


In [14]:
f = (pt / ptr) ** (1 / r) - 1
print(f"f({t},{r}) = {f*100:.2f}%")

f(3,2) = 10.95%


第二種：[綠角財經筆記](http://greenhornfinancefootnote.blogspot.com/2010/08/how-to-compute-forward-rates-from.html)  
-> 使用者輸入各期 Spot Rate， 計算 Forward Rate

1. 首先要知道總共持續多少時間與多久計算一次利息，才有辦法決定接下來要計算多少資

In [15]:
n = float(input("Years to Maturity: "))
m = int(input("Payment(1. Annually, 2. Semi-annually, 4. Quarterly): "))

Years to Maturity:  1.5
Payment(1. Annually, 2. Semi-annually, 4. Quarterly):  2


2. 使用者輸入各期 Spot Rate

In [16]:
spot_rate_list = [0.0]
for i in range(int(m * n)):
    spot_rate = float(input(f"Add the spot rate(%) of {(i+1)/m} year zero coupon bond: ")) / 100
    spot_rate_list.append(spot_rate)

Add the spot rate(%) of 0.5 year zero coupon bond:  1
Add the spot rate(%) of 1.0 year zero coupon bond:  2
Add the spot rate(%) of 1.5 year zero coupon bond:  3.03


3. 定義 Forward Rate 計算 function

In [17]:
def compute_forward(spot_rate_list):
    forward_rate_matrix = list()
    for i in range(len(spot_rate_list)):
        forward_rates = list()
        for j in range(len(spot_rate_list)):
            if j <= i:
                forward_rates.append("x")
            else:
                forward_rate = (((1 + spot_rate_list[j]) ** j) / ((1 + spot_rate_list[i]) ** i)) ** (1 / (j - i)) - 1
                forward_rates.append(f"{round(forward_rate*100, 2)}%")
        forward_rate_matrix.append(forward_rates)
    return forward_rate_matrix

4. 輸出表格

In [18]:
import pandas as pd
from tabulate import tabulate

label = [i / m for i in range(len(spot_rate_list))]
forward_rate_table = pd.DataFrame(compute_forward(spot_rate_list), columns=label, index=label)

print("--- Forward Rate Table ---")
print(tabulate(forward_rate_table, headers='keys', tablefmt="grid"))

--- Forward Rate Table ---
+-----+-------+-------+-------+-------+
|     | 0.0   | 0.5   | 1.0   | 1.5   |
+=====+=======+=======+=======+=======+
| 0   | x     | 1.0%  | 2.0%  | 3.03% |
+-----+-------+-------+-------+-------+
| 0.5 | x     | x     | 3.01% | 4.06% |
+-----+-------+-------+-------+-------+
| 1   | x     | x     | x     | 5.12% |
+-----+-------+-------+-------+-------+
| 1.5 | x     | x     | x     | x     |
+-----+-------+-------+-------+-------+
